In [1]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='west0_ws', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4048 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0_ws...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:23


In [211]:
sphynx.stop()

In [2]:
# Ingame BP 획득
target_date = "2022-07-20"
mapdata = load_schema.ingame(spark, "pc", "live", "LogMatchEnd", target_date, target_date)

In [3]:
start_date = "2022-07-13"
end_date = "2022-07-19"
all_gcoin = load_data_mart('pc', start_date, end_date, "gcoin_use")
gcoin = load_data_mart('pc', start_date, end_date, "gcoin_use").where(col("event_name").like("%workshop%"))

In [4]:
gcoin_master = load_data_mart('pc', end_date, end_date, "gcoin_master")

In [5]:
gcoin = gcoin.join(gcoin_master.select("account_id", "first_use_date"), "account_id", "left")

In [166]:
from pyspark.sql.types import * 

def classify_country(country_os, country_ip):
    if country_os != 'CN':
        return country_ip
    else:
        return country_os

country_type_udf = udf(classify_country, StringType())

meta_region = mysql.read_table(spark, 'metainfo', 'meta_bi_regions')

user = load_data_mart('pc', end_date, end_date, "user_master")
user = user.withColumn("country_new", country_type_udf("country_os", "country_ip"))
user = user.join(meta_region, user.country_new == meta_region.country_code_iso2, "left")

In [167]:
user = user.withColumnRenamed("accountid", "account_id")

In [168]:
gcoin = gcoin.join(user.select("account_id", "pubg_region"), "account_id", "left")

In [8]:
user.select(countDistinct("accountid").alias("au")).show(truncate=False)

+-------+
|au     |
+-------+
|4671695|
+-------+



In [13]:
gcoin.withColumn("paid_pu", when(col("paid_use")>0, col("account_id")).otherwise(lit(None))) \
    .select(countDistinct("account_id").alias("total_pu"), countDistinct("paid_pu").alias("paid_pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show()

+--------+-------+---------+----------+
|total_pu|paid_pu| paid_use| total_use|
+--------+-------+---------+----------+
|  317870| 144304|906826030|1112161360|
+--------+-------+---------+----------+



In [111]:
gcoin.where(col("date") == col("first_use_date")).select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                     39086|
+--------------------------+



In [13]:
npu = gcoin.where(col("first_use_date")== col("date")).select("account_id").distinct()

In [7]:
gcoin.where(col("date") == col("first_use_date")).withColumn("paid_account_id", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None))) \
    .select(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), countDistinct("paid_account_id").alias("paid_pu")).show(truncate=False)

+--------+---------+-------+
|paid_use|total_use|paid_pu|
+--------+---------+-------+
|6882690 |30076570 |1745   |
+--------+---------+-------+



In [14]:
gcoin.join(npu, "account_id") \
    .select(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).show(truncate=False)

+--------+---------+
|paid_use|total_use|
+--------+---------+
|10637360|35688250 |
+--------+---------+



In [15]:
# 전체 total gcoin중 npu의 total gcoin 차지율
35688250/1112161360*100

3.2089093618573474

In [16]:
# 전체 paid gcoin중 npu의 paid gcoin 차지율
10637360/906826030*100

1.1730320533476526

In [12]:
6882690/30076570*100

22.883892677921718

In [8]:
6882690/30076570*100

22.883892677921718

In [11]:
1745/39086*100

4.46451414828839

In [214]:
gcoin.withColumn("paid_pu", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None))) \
    .groupBy("product_name").agg(countDistinct("account_id").alias("total_pu"), countDistinct("paid_pu").alias("paid_pu"), sum("paid_use").alias("paid_use"), sum("free_use").alias('free_use')).toPandas()

,product_name,total_pu,paid_pu,paid_use,free_use
0,ARCHIVIST'S CHEST BUNDLE (x10),59606,58470,598093120,38595680
1,ARCHIVIST'S CHEST,38460,16722,4196005,6918815
2,HUNTER'S CHEST PACK (x1),50107,22391,11453960,13144200
3,HUNTER'S CHEST,56172,18903,2973745,6284015
4,HUNTER'S CHEST PACK (x5),26172,22106,169382285,20796315
5,ARCHIVIST'S CHEST BUNDLE (x1),136371,63389,35153640,50686680
6,ARCHIVIST'S CHEST BUNDLE (x5),47886,34989,50409710,35281090
7,KEY,151662,67777,35163565,33628535


In [12]:
user.groupBy("pubg_region").agg(countDistinct("account_id")).toPandas()

,pubg_region,count(DISTINCT account_id)
0,CN,2782808
1,NA,157589
2,SA,119825
3,KR,625235
4,SEA,225167
5,Undefined,242
6,JP,21326
7,TW/HK,33197
8,CIS,306999
9,EMEA,375760


In [13]:
gcoin.groupBy("pubg_region").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).toPandas()

,pubg_region,paid_use,total_use
0,CN,758296570,910113330
1,NA,12770530,17718620
2,SA,2368470,5420090
3,KR,88419990,110549060
4,SEA,8313790,12022330
5,Undefined,0,380
6,JP,1421600,1870970
7,TW/HK,4598970,5672780
8,CIS,8041090,14774940
9,EMEA,19267540,29703350


In [15]:
gcoin.groupBy("pubg_region").agg(countDistinct("account_id").alias("total_pu")).toPandas()

,pubg_region,total_pu
0,CN,210842
1,NA,9062
2,SA,7259
3,KR,35390
4,SEA,9320
5,Undefined,2
6,JP,797
7,TW/HK,1552
8,CIS,18240
9,EMEA,23537


In [16]:
gcoin.where(col("paid_use") > 0).groupBy("pubg_region").agg(countDistinct("account_id").alias("paid_pu")).toPandas()

,pubg_region,paid_pu
0,CN,103192
1,NA,3528
2,SA,1847
3,KR,17928
4,SEA,2990
5,JP,305
6,TW/HK,750
7,CIS,4995
8,EMEA,7849
9,OC,920


In [112]:
gcoin.where(col("date") == col("first_use_date")).groupBy("pubg_region").agg(countDistinct("account_id").alias("npu")).toPandas()

,pubg_region,npu
0,CN,21589
1,NA,2410
2,SA,1133
3,KR,3263
4,SEA,1318
5,Undefined,2
6,JP,197
7,TW/HK,184
8,CIS,4444
9,EMEA,4236


In [54]:
by_region_product = gcoin.withColumn("paid_pu", when(col("paid_use") > 0, col("account_id")).otherwise(lit(None))) \
    .groupBy("pubg_region", "product_name") \
    .agg(countDistinct("account_id").alias("pu"), countDistinct("paid_pu").alias("paid_pu"), sum("qty").alias("unit_sold"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).toPandas()

In [55]:
by_region_product.to_csv("./by_region_product.csv", index=False)

In [23]:
user.groupBy("platform").agg(countDistinct("account_id").alias("au")).toPandas()

,platform,au
0,KAKAO,283838
1,STEAM,4387857


In [20]:
gcoin.groupBy("platform").agg(countDistinct("account_id").alias("pu")).toPandas()

,platform,pu
0,KAKAO,15800
1,STEAM,302070


In [21]:
gcoin.where(col("paid_use") > 0).groupBy("platform").agg(countDistinct("account_id").alias("pu")).toPandas()

,platform,pu
0,KAKAO,7900
1,STEAM,136404


In [22]:
gcoin.groupBy("platform").agg(sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use")).toPandas()

,platform,paid_use,total_use
0,KAKAO,39942470,50057260
1,STEAM,866883560,1062104100


In [169]:
purchase = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date)

In [101]:
# 18.1 update 후 일주일
pre_start_date = "2022-06-08"
pre_end_date = "2022-06-14"
pre_purchase = load_schema.lobby(spark, "pc", "live", "PurchaseResult", pre_start_date, pre_end_date).where(col('Currency').like("%bp%"))

In [102]:
pre_bp_purchase = pre_purchase.groupBy("AnalyticEventType", "ProductId").agg(countDistinct("AccountId").alias("pu"), sum("LocalPrice").alias("amount")).toPandas()
pre_bp_purchase.to_csv("./pre_bp_purchase.csv", index=False)

In [103]:
pre_purchase.groupBy("AnalyticEventType").agg(countDistinct("AccountId").alias("pu"), sum("LocalPrice").alias("amount")).toPandas()

,AnalyticEventType,pu,amount
0,other,7904,251615000
1,wsus,1558560,2169178400
2,other,732612,3449759100
3,brand,10109,109015000
4,holiday,4534,25535000


In [132]:
pre_purchase.withColumn("EventType", when(col("AnalyticEventType").like("%other%"), lit("other")).otherwise(col("AnalyticEventType"))) \
    .groupBy("EventType").agg(countDistinct("AccountId").alias("pu"), sum("LocalPrice").alias("amount")).toPandas()

,EventType,pu,amount
0,wsus,1558560,2169178400
1,other,736395,3701374100
2,brand,10109,109015000
3,holiday,4534,25535000


In [58]:
purchase.where((col('Currency').like("%bp%")) & (col("AnalyticEventType") == "workshop")) \
    .groupBy("AccountId").agg(countDistinct("ProductId").alias("product_cnt")) \
    .groupBy("product_cnt").agg(countDistinct("AccountId")).show()

+-----------+-------------------------+
|product_cnt|count(DISTINCT AccountId)|
+-----------+-------------------------+
|          1|                   327191|
|          2|                   591621|
+-----------+-------------------------+



In [135]:
bp_purchase = purchase.where(col('Currency').like("%bp%")).groupBy("AnalyticEventType", "ProductId").agg(countDistinct("AccountId").alias("pu"), sum("LocalPrice").alias("amount")).toPandas()

In [139]:
bp_purchase.to_csv("./bp_purchase.csv", index=False)

In [137]:
bp_purchase[bp_purchase.AnalyticEventType == "workshop"]

,AnalyticEventType,ProductId,pu,amount
17,workshop,itemdesc.14300001,756877,7064690000
25,workshop,currencydesc.chestkey,753556,5517012000


In [38]:
purchase.where(col('Currency').like("%bp%")).groupBy("AnalyticEventType").agg(countDistinct("AccountId").alias("pu"), sum("LocalPrice").alias("amount")).toPandas()

,AnalyticEventType,pu,amount
0,other,4320,143405000
1,wsus,1282922,1730712000
2,other,15899,123524300
3,brand,5652,57484000
4,workshop,918812,12581702000
5,holiday,2726,14840000


In [133]:
purchase.where(col('Currency').like("%bp%")).withColumn("EventType", when(col("AnalyticEventType").like("%other%"), lit("other")).otherwise(col("AnalyticEventType"))) \
    .groupBy("EventType").agg(countDistinct("AccountId").alias("pu"), sum("LocalPrice").alias("amount")).toPandas()

,EventType,pu,amount
0,wsus,1282922,1730712000
1,other,19880,266929300
2,brand,5652,57484000
3,workshop,918812,12581702000
4,holiday,2726,14840000


In [45]:
workshop_bp_account_id = purchase.where((col("Currency").like("%bp%")) & (col("AnalyticEventType") == "workshop")).select(col("AccountId")).distinct().toPandas()["AccountId"]

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38377)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38377)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

In [40]:
# workshop_bp_account_id.to_csv("./workshop_bp_account_id.csv", index=False)
workshop_bp_account_id = pd.read_csv("./workshop_bp_account_id.csv")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [43]:
len(workshop_bp_account_id)

918812

In [45]:
gcoin.where(col("account_id").isin(workshop_bp_account_id.to_list())).select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                    233536|
+--------------------------+



In [62]:
gcoin.where((col("paid_use") > 0) & (col("account_id").isin(workshop_bp_account_id.to_list()))).select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                    119482|
+--------------------------+



In [109]:
gcoin.where((col("account_id").isin(workshop_bp_account_id.to_list())) & (col("date") == col("first_use_date"))).select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                     21557|
+--------------------------+



In [59]:
bp_workshop_first_use = purchase.where((col("Currency").like("%bp%")) & (col("AnalyticEventType") == "workshop")).groupBy("AccountId").agg(min("Time").alias("first_date"))
gcoin_npu = gcoin.where(col("date") == col("first_use_date")).groupBy("account_id").agg(min("time").alias("gcoin_first_date"))
gcoin_after_bp = bp_workshop_first_use.join(gcoin_npu, (bp_workshop_first_use.AccountId == gcoin_npu.account_id) & (bp_workshop_first_use.first_date <= gcoin_npu.gcoin_first_date))

In [115]:
# bp 로 workshop 상품 구매 후 gcoin으로 처음 상품 구매
gcoin_after_bp.count()

18595

In [62]:
gcoin_after_bp_user = gcoin_after_bp.toPandas()["AccountId"].to_list()
gcoin.where(col("account_id").isin(gcoin_after_bp_user)).groupBy("product_name").agg(countDistinct("account_id").alias("pu"), sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use")).toPandas()

,product_name,pu,free_use,paid_use
0,ARCHIVIST'S CHEST BUNDLE (x10),397,558450,1555950
1,ARCHIVIST'S CHEST,1147,335650,13010
2,HUNTER'S CHEST PACK (x1),2066,637970,49260
3,HUNTER'S CHEST,1289,229340,10900
4,HUNTER'S CHEST PACK (x5),466,462900,436750
5,ARCHIVIST'S CHEST BUNDLE (x1),5094,2365540,212060
6,ARCHIVIST'S CHEST BUNDLE (x5),1929,2274500,240700
7,KEY,5523,1060360,115440


In [64]:
gcoin.where((col("account_id").isin(workshop_bp_account_id.to_list()))).groupBy("product_name") \
    .agg(countDistinct("account_id").alias("pu"), sum("qty").alias("qty"), sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use")).toPandas()

,product_name,pu,qty,free_use,paid_use
0,ARCHIVIST'S CHEST BUNDLE (x10),52556,242336,34137030,547469370
1,ARCHIVIST'S CHEST,32325,54266,5911150,3856730
2,None,123258,630235,29904745,33118755
3,HUNTER'S CHEST PACK (x1),37388,110783,10428180,10620590
4,HUNTER'S CHEST,42784,62128,4838495,2616865
5,HUNTER'S CHEST PACK (x5),23348,189194,18580830,161153470
6,ARCHIVIST'S CHEST BUNDLE (x1),102217,296161,39557470,31521170
7,ARCHIVIST'S CHEST BUNDLE (x5),39749,61931,28180750,46136450


In [65]:
gcoin.where((col("account_id").isin(workshop_bp_account_id.to_list())) & (col("paid_use") > 0)).groupBy("product_name") \
    .agg(countDistinct("account_id").alias("pu"), sum("qty").alias("qty"), sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use")).toPandas()

,product_name,pu,qty,free_use,paid_use
0,ARCHIVIST'S CHEST BUNDLE (x10),51637,240081,28725030,547469370
1,ARCHIVIST'S CHEST,14943,27276,1052950,3856730
2,None,59856,405417,7422945,33118755
3,HUNTER'S CHEST PACK (x1),19191,70708,2813930,10620590
4,HUNTER'S CHEST,15585,27317,661175,2616865
5,HUNTER'S CHEST PACK (x5),20339,181973,11720880,161153470
6,ARCHIVIST'S CHEST BUNDLE (x1),53561,168793,8989150,31521170
7,ARCHIVIST'S CHEST BUNDLE (x5),31124,51243,15355150,46136450


In [66]:
gcoin.where((col("account_id").isin(workshop_bp_account_id.to_list())) & (col("free_use") > 0)).groupBy("product_name") \
    .agg(countDistinct("account_id").alias("pu"), sum("qty").alias("qty"), sum("free_use").alias("free_use"), sum("paid_use").alias("paid_use")).toPandas()

,product_name,pu,qty,free_use,paid_use
0,ARCHIVIST'S CHEST BUNDLE (x10),39770,87759,34137030,176484570
1,ARCHIVIST'S CHEST,27463,42123,5911150,1670990
2,None,108771,413191,29904745,11414355
3,HUNTER'S CHEST PACK (x1),32864,84427,10428180,5612950
4,HUNTER'S CHEST,37970,49689,4838495,1124185
5,HUNTER'S CHEST PACK (x5),18260,92059,18580830,68875220
6,ARCHIVIST'S CHEST BUNDLE (x1),91478,217996,39557470,12761570
7,ARCHIVIST'S CHEST BUNDLE (x5),33096,42720,28180750,23083250


In [170]:
open_crate = load_schema.lobby(spark, "pc", "live", "WorkshopCrateOpened", start_date, end_date)

In [171]:
craft = load_schema.lobby(spark, "pc", "live", "WorkshopCrafted", start_date, end_date)

In [172]:
disassemble = load_schema.lobby(spark, "pc", "live", "WorkshopDisassembled", start_date, end_date)

In [173]:
repurpose = load_schema.lobby(spark, "pc", "live", "WorkshopRepurposed", start_date, end_date)

In [174]:
special_craft = load_schema.lobby(spark, "pc", "live", "PurchaseResult", start_date, end_date).where(col("Currency") == "artisanstoken")

In [175]:
open_crate = open_crate.join(user.withColumnRenamed("account_id", "AccountId"), "AccountId")
craft = craft.join(user.withColumnRenamed("account_id", "AccountId"), "AccountId")
disassemble = disassemble.join(user.withColumnRenamed("account_id", "AccountId"), "AccountId")
repurpose = repurpose.join(user.withColumnRenamed("account_id", "AccountId"), "AccountId")
special_craft = special_craft.join(user.withColumnRenamed("account_id", "AccountId"), "AccountId")
bp_workshop = purchase.where((col("Currency").like("%bp%")) & (col("AnalyticEventType") == "workshop")).join(user.withColumnRenamed("account_id", "AccountId"), "AccountId")

In [176]:
# workshop_user = (open_crate.groupBy("pubg_region", "AccountId").agg(sum("OpenAmount").alias("amount")).withColumn("action", lit("open"))) \
#     .unionByName(craft.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("craft"))) \
#     .unionByName(disassemble.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("disassemble"))) \
#     .unionByName(repurpose.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("repurpose"))) \
#     .unionByName(special_craft.groupBy("pubg_region", "AccountId").agg(sum("Amount").alias("amount")).withColumn("action", lit("special_craft"))) \
#     .unionByName(gcoin.groupBy("pubg_region", "account_id").agg(sum("qty").alias("amount")).withColumnRenamed("account_id", "AccountId").withColumn("action", lit("gcoin"))) \
#     .unionByName(bp_workshop.groupBy("pubg_region", "AccountId").agg(count("*").alias("amount")).withColumn("action", lit("bp_workshop")))
workshop_user = (open_crate.groupBy("AccountId").agg(sum("OpenAmount").alias("amount")).withColumn("action", lit("open"))) \
    .unionByName(craft.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("craft"))) \
    .unionByName(disassemble.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("disassemble"))) \
    .unionByName(repurpose.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("repurpose"))) \
    .unionByName(special_craft.groupBy("AccountId").agg(sum("Amount").alias("amount")).withColumn("action", lit("special_craft"))) \
    .unionByName(gcoin.groupBy("account_id").agg(sum("qty").alias("amount")).withColumnRenamed("account_id", "AccountId").withColumn("action", lit("gcoin"))) \
    .unionByName(bp_workshop.groupBy("AccountId").agg(count("*").alias("amount")).withColumn("action", lit("bp_workshop")))

In [177]:
workshop_user_list = workshop_user.select("AccountId").distinct()

In [195]:
# yes_workshop_user_gcoin = all_gcoin.join(workshop_user_list.withColumnRenamed("AccountId", "account_id"), "account_id")

In [199]:
all_gcoin = all_gcoin.withColumn("type", when(~col("event_type").isin(["workshop", "wsus", "survivorpass"]), lit("other")).otherwise(col("event_type"))) \
    .groupBy("account_id", "type").agg(count("*").alias("cnt"))
all_gcoin_df = all_gcoin \
    .withColumn("workshop", when(col("type") == "workshop", lit(1)).otherwise(lit(0))) \
    .withColumn("wsus", when(col("type") == "wsus", lit(1)).otherwise(lit(0))) \
    .withColumn("survivorpass", when(col("type") == "survivorpass", lit(1)).otherwise(lit(0))) \
    .withColumn("store", when(col("type") == "other", lit(1)).otherwise(lit(0)))

In [200]:
all_gcoin_df = all_gcoin_df.groupBy("account_id").agg( \
                             sum("workshop").alias("workshop"), sum("wsus").alias("wsus"), sum("survivorpass").alias("survivorpass"), sum("store").alias("store"))

all_gcoin_df = all_gcoin_df.withColumn("group", concat(col("workshop"), lit(","), col("wsus"), lit(","), col("survivorpass"), lit(","), col("store")))

In [208]:
all_gcoin.select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                    587316|
+--------------------------+



In [201]:
all_gcoin_df.groupBy("group").agg(countDistinct("account_id").alias("user_cnt")).toPandas()

,group,user_cnt
0,"1,1,0,1",3425
1,"1,0,0,0",190147
2,"1,1,1,0",12754
3,"1,0,0,1",13531
4,"0,1,0,0",93712
5,"0,0,1,1",6344
6,"0,1,0,1",8831
7,"0,0,0,0",9902
8,"1,0,1,0",52658
9,"0,1,1,1",1658


In [202]:
no_workshop_user_gcoin = all_gcoin.join(workshop_user_list.withColumnRenamed("AccountId", "account_id"), "account_id", "leftanti")

In [203]:
no_workshop_user_gcoin.groupBy("event_name", "event_type").agg(countDistinct("account_id").alias("pu"), sum("paid_use").alias("paid_use"), sum("free_use").alias("free_use")).orderBy("pu", ascending=False).toPandas()

AnalysisException: "cannot resolve '`event_name`' given input columns: [account_id, type, cnt];;\n'Aggregate ['event_name, 'event_type], ['event_name, 'event_type, count(distinct account_id#2) AS pu#4133L, sum('paid_use) AS paid_use#4135, sum('free_use) AS free_use#4137]\n+- Project [account_id#2, type#3972, cnt#4045L]\n   +- Join LeftAnti, (account_id#2 = account_id#4124)\n      :- Aggregate [account_id#2, type#3972], [account_id#2, type#3972, count(1) AS cnt#4045L]\n      :  +- Project [date#0, platform#1, account_id#2, product_id#3, free_use#5L, free_balance#6L, paid_use#7L, paid_balance#8L, price#9L, qty#10L, reason#11, time#12, transaction_id#13, gcoin_by_expiry#14, CASE WHEN NOT event_type#42 IN (workshop,wsus,survivorpass) THEN other ELSE event_type#42 END AS type#3972, is_salesid_exist#135, sales_id_#153, device#32, currency#34, sales_id#35, exchange_id#37, product_name#38, product_type#39, category#40, ... 11 more fields]\n      :     +- Project [date#0, platform#1, account_id

In [ ]:
no_workshop_user_gcoin.withColumn("type", when(col("event_type").like("%other%"), lit("other")).otherwise(col("event_type"))) \
    .groupBy("type").agg(countDistinct("account_id").alias("pu"), sum("paid_use").alias("paid_use"), sum("free_use").alias("free_use")).orderBy("pu", ascending=False).toPandas()

In [156]:
no_workshop_user_gcoin.select(countDistinct("account_id").alias("no workshop user but gcoin user count")).show()

+-------------------------------------+
|no workshop user but gcoin user count|
+-------------------------------------+
|                                92962|
+-------------------------------------+



In [81]:
# workshop_user = (open_crate.withColumn("action", lit("open")).select("AccountId", "action", "pubg_region").distinct()) \
#     .union((craft.withColumn("action", lit("craft")).select("AccountId", "action", "pubg_region").distinct())) \
#     .union((disassemble.withColumn("action", lit("disassemble")).select("AccountId", "action", "pubg_region").distinct())) \
#     .union((repurpose.withColumn("action", lit("repurpose")).select("AccountId", "action", "pubg_region").distinct())) \
#     .union((special_craft.withColumn("action", lit("special_craft")).select("AccountId", "action", "pubg_region").distinct())) \
#     .union((gcoin.withColumn("action", lit("purchase")).withColumnRenamed("account_id", "AccountId").select("AccountId", "action", "pubg_region").distinct()))

In [46]:
# gcoin purchase, open, craft, disassemble, repurpose, special_craft
workshop_user.select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                  1441549|
+-------------------------+



In [155]:
# gcoin purchase, open, craft, disassemble, repurpose, special_craft, bp workshop purchase
workshop_user.select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                  1547059|
+-------------------------+



In [93]:
workshop_user.groupBy("action").agg(countDistinct("AccountId").alias("user_cnt"), sum("amount").alias("amount")).toPandas()

,action,user_cnt,amount
0,disassemble,449640,1461980
1,repurpose,80945,505132
2,craft,478364,1800017
3,gcoin,317870,1850834
4,special_craft,87161,415065
5,open,1423939,10120769


In [156]:
workshop_user.groupBy("action").agg(countDistinct("AccountId").alias("user_cnt"), sum("amount").alias("amount")).toPandas()

,action,user_cnt,amount
0,disassemble,449640,1461980
1,repurpose,80945,505132
2,bp_workshop,918812,2792191
3,craft,478364,1800017
4,gcoin,317870,1850834
5,special_craft,87161,415065
6,open,1423939,10120769


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 58000)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.6/socketserver.py", line 721, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 269, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 241, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 245, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/pysp

In [120]:
workshop_action_by_region = workshop_user.groupBy("action", "pubg_region").agg(countDistinct("AccountId").alias("user_cnt"), sum("amount").alias("amount"))
workshop_user_by_region = workshop_user.groupBy("pubg_region").agg(countDistinct("AccountId").alias("all_user_cnt"))
workshop_by_region = workshop_action_by_region.join(workshop_user_by_region, "pubg_region", "left").toPandas()
workshop_by_region.to_csv("./workshop_by_region.csv", index=False)

In [124]:
workshop_by_region.groupby('pubg_region', as_index=False)['all_user_cnt'].mean()

,pubg_region,all_user_cnt
0,CIS,101295
1,CN,959179
2,EMEA,115296
3,JP,4601
4,KR,202823
5,NA,36017
6,OC,7979
7,SA,37555
8,SEA,71627
9,TW/HK,10682


In [ ]:
workshop_user_df = workshop_user.withColumn("gcoin", when(col("action") == "gcoin", lit(1)).otherwise(lit(0))) \
    .withColumn("open", when(col("action") == "open", lit(1)).otherwise(lit(0))) \
    .withColumn("disassemble", when(col("action") == "disassemble", lit(1)).otherwise(lit(0))) \
    .withColumn("repurpose", when(col("action") == "repurpose", lit(1)).otherwise(lit(0))) \
    .withColumn("craft", when(col("action") == "craft", lit(1)).otherwise(lit(0))) \
    .withColumn("special_craft", when(col("action") == "special_craft", lit(1)).otherwise(lit(0))) \
    .withColumn("bp_workshop", when(col("action") == "bp_workshop", lit(1)).otherwise(lit(0)))

workshop_user_df = workshop_user_df.groupBy("AccountId").agg(sum("gcoin").alias("gcoin"), \
                                          sum("open").alias("open"), \
                                          sum("disassemble").alias("disassemble"), \
                                          sum("repurpose").alias("repurpose"), \
                                          sum("craft").alias("craft"), \
                                          sum("special_craft").alias("special_craft"), \
                                          sum("bp_workshop").alias("bp_workshop"))

workshop_user_df = workshop_user_df.withColumn("group", concat(col("bp_workshop"), col("gcoin"), col("open"), col("disassemble"), col("repurpose"), col("craft"), col("special_craft")))

In [53]:
workshop_user_df = workshop_user.withColumn("gcoin", when(col("action") == "gcoin", lit(1)).otherwise(lit(0))) \
    .withColumn("open", when(col("action") == "open", lit(1)).otherwise(lit(0))) \
    .withColumn("disassemble", when(col("action") == "disassemble", lit(1)).otherwise(lit(0))) \
    .withColumn("repurpose", when(col("action") == "repurpose", lit(1)).otherwise(lit(0))) \
    .withColumn("craft", when(col("action") == "craft", lit(1)).otherwise(lit(0))) \
    .withColumn("special_craft", when(col("action") == "special_craft", lit(1)).otherwise(lit(0))) \
    .withColumn("bp_workshop", when(col("action") == "bp_workshop", lit(1)).otherwise(lit(0)))

In [54]:
workshop_user_df = workshop_user_df.groupBy("AccountId").agg(sum("gcoin").alias("gcoin"), \
                                          sum("open").alias("open"), \
                                          sum("disassemble").alias("disassemble"), \
                                          sum("repurpose").alias("repurpose"), \
                                          sum("craft").alias("craft"), \
                                          sum("special_craft").alias("special_craft"), \
                                          sum("bp_workshop").alias("bp_workshop"))

In [55]:
workshop_user_df = workshop_user_df.withColumn("group", concat(col("bp_workshop"), col("gcoin"), col("open"), col("disassemble"), col("repurpose"), col("craft"), col("special_craft")))

In [56]:
only_open_user = workshop_user_df.where(col("group") == "0010000").select("AccountId").toPandas()["AccountId"]
workshop_crate_increased = load_schema.lobby(spark, "pc", "live", "WorkshopCrateIncreased", start_date, end_date)
workshop_crate_increased.where(col("AccountId").isin(only_open_user.to_list())).groupBy("Reason").agg(sum("Amount")).show(truncate=False)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38377)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:38377)
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception

+---------------------------+-----------+
|Reason                     |sum(Amount)|
+---------------------------+-----------+
|buy-cash-item              |9          |
|battlepass-mission-reward  |12805      |
|outgameevent-mission-reward|755488     |
|open-crate-bonus           |46291      |
|battlepass-level-reward    |20288      |
|open-crate                 |-663263    |
+---------------------------+-----------+



In [30]:
workshop_user_df.groupBy("group").agg(countDistinct("AccountId").alias("user_cnt")).orderBy("user_cnt", ascending=False).toPandas()

,group,user_cnt
0,0010000,419373
1,1010000,287617
2,1000000,105510
3,1011010,103127
4,1010010,82690
5,1011000,75197
6,1111010,73489
7,0010010,64719
8,0011000,45959
9,0110000,36266


In [47]:
# gcoin purchase, open, craft, disassemble, repurpose, special_craft
workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).groupBy("action_cnt").agg(countDistinct("AccountId").alias("user_cnt")).toPandas()

,action_cnt,user_cnt
0,6,37746
1,5,44684
2,1,724544
3,3,182889
4,2,345966
5,4,105720


In [207]:
# bp purchase, gcoin purchase, open, craft, disassemble, repurpose, special_craft
workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).groupBy("action_cnt").agg(countDistinct("AccountId").alias("user_cnt")).toPandas()

,action_cnt,user_cnt
0,7,35389
1,6,43332
2,5,97675
3,1,536348
4,3,228603
5,2,442182
6,4,163530


In [31]:
action_1_user = workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).where(col("action_cnt") == 1)
action_2_user = workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).where(col("action_cnt") == 2)
action_3_user = workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).where(col("action_cnt") == 3)
action_4_user = workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).where(col("action_cnt") == 4)
action_5_user = workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).where(col("action_cnt") == 5)

In [209]:
workshop_user.join(action_1_user, "AccountId").groupBy("action").agg(countDistinct("AccountId")).show()

+-----------+-------------------------+
|     action|count(DISTINCT AccountId)|
+-----------+-------------------------+
|bp_workshop|                   105510|
|      gcoin|                    11465|
|       open|                   419373|
+-----------+-------------------------+



In [32]:
workshop_user.join(action_2_user, "AccountId").groupBy("action").agg(countDistinct("AccountId")).show()

+-------------+-------------------------+
|       action|count(DISTINCT AccountId)|
+-------------+-------------------------+
|  disassemble|                    45959|
|    repurpose|                      129|
|  bp_workshop|                   293706|
|        craft|                    64719|
|        gcoin|                    42382|
|special_craft|                     1403|
|         open|                   436066|
+-------------+-------------------------+



In [33]:
workshop_user.join(action_3_user, "AccountId").groupBy("action").agg(countDistinct("AccountId")).show()

+-------------+-------------------------+
|       action|count(DISTINCT AccountId)|
+-------------+-------------------------+
|  disassemble|                    94679|
|    repurpose|                     5469|
|  bp_workshop|                   197490|
|        craft|                   104026|
|        gcoin|                    51926|
|special_craft|                     3645|
|         open|                   228574|
+-------------+-------------------------+



In [20]:
# open, craft, disassemble, repurpose, special_craft
workshop_user.select(countDistinct("AccountId")).show()

+-------------------------+
|count(DISTINCT AccountId)|
+-------------------------+
|                  1423995|
+-------------------------+



workshop action 몇가지를 했는지

In [32]:
# open, craft, disassemble, repurpose, special_craft
workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).groupBy("action_cnt").agg(countDistinct("AccountId").alias("user_cnt")).toPandas()

,action_cnt,user_cnt
0,5,38800
1,1,775899
2,3,214288
3,2,336373
4,4,58635


In [37]:
# open외에 다른 행동은 무엇을 하는가
action_2_user = workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).where(col("action_cnt") == 2)
action_3_user = workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).where(col("action_cnt") == 3)
action_4_user = workshop_user.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).where(col("action_cnt") == 4)

In [38]:
workshop_user.join(action_2_user.select("AccountId").distinct(), "AccountId").groupBy("action").agg(countDistinct("AccountId")).show()

+-------------+-------------------------+
|       action|count(DISTINCT AccountId)|
+-------------+-------------------------+
|  disassemble|                   148698|
|    repurpose|                     7845|
|        craft|                   175399|
|special_craft|                     4431|
|         open|                   336373|
+-------------+-------------------------+



In [30]:
not_open_crate = workshop_user.join(open_crate.select("AccountId").distinct(), "AccountId", "leftanti")

In [34]:
not_open_crate.show(60, truncate=False)

+----------------------------------------+-------------+
|AccountId                               |action       |
+----------------------------------------+-------------+
|account.7b5c5abfe2fb41dd85072090febdb7dd|special_craft|
|account.07fb4d320074457694025466ae693ec3|special_craft|
|account.42b2cc2f2a5545c8938453be7e60a76b|special_craft|
|account.ff54ee01588d4f0a8ef42feaad97317d|special_craft|
|account.0952eab5dfd14eedb1217a162ec8e4cf|special_craft|
|account.403d0680d51d4babbdeff3eb2e952f58|special_craft|
|account.c7a2c3144cad4f22976c117cb721a8d4|special_craft|
|account.a53b7d5fd58742a082f1707c08896d96|special_craft|
|account.6bb2026b2aa14d9d91c45893e3ee68d4|special_craft|
|account.ccba98b6186a4a03a7ab5e1ce90408d3|special_craft|
|account.7c018e74571047b993d1527a7502ce0d|special_craft|
|account.2baf70c782d24b389c60adc396186afc|special_craft|
|account.ecd5658049ca41acb8b804a6081f6657|special_craft|
|account.147525f755db47a39f68939be71045c4|special_craft|
|account.8e72f2fcd7414d3d91937f

In [35]:
no_open_crate_action = workshop_user.join(not_open_crate.select("AccountId").distinct(), "AccountId")

In [36]:
no_open_crate_action.groupBy("AccountId").agg(countDistinct("action").alias("action_cnt")).groupBy("action_cnt").agg(countDistinct("AccountId").alias("user_cnt")).toPandas()

,action_cnt,user_cnt
0,1,56


In [40]:
gcoin.join(no_open_crate_action.withColumnRenamed("AccountId", "account_id"), "account_id") \
    .groupBy("product_name").agg(countDistinct("account_id").alias("pu"), sum("paid_use").alias("paid_use"), sum(col("paid_use") + col("free_use")).alias("total_use"), sum("qty").alias("qty")).toPandas()

,product_name,pu,paid_use,total_use,qty
0,ARCHIVIST'S CHEST BUNDLE (x10),52,165510,180000,75
1,ARCHIVIST'S CHEST,11,2830,3600,20
2,None,17,3800,5500,55
3,HUNTER'S CHEST PACK (x1),5,330,1140,6
4,HUNTER'S CHEST,13,2080,3120,26
5,HUNTER'S CHEST PACK (x5),5,5010,5700,6
6,ARCHIVIST'S CHEST BUNDLE (x1),17,6070,11040,46
7,ARCHIVIST'S CHEST BUNDLE (x5),4,4860,6000,5


In [42]:
special_craft.join(not_open_crate.select("AccountId").distinct(), "AccountId").groupBy("ProductId")\
    .agg(sum("LocalPrice").alias("token_used"), countDistinct("AccountId").alias("pu"), sum("Amount").alias("amount")).toPandas()

,ProductId,token_used,pu,amount
0,itemdesc.13000579,610,54,61
1,itemdesc.13000612,100,1,1
2,itemdesc.13000611,100,1,1
